In [1]:
import numpy as np
import pandas as pd
import pandapower as pp


#Create an empty network
net = pp.create_empty_network(f_hz=60) 

In [2]:

p_demand = 1080e-6
q_demand = 923.6e-6



In [3]:
#creating buses
central_generation = pp.create_bus(net, vn_kv = 69.0, name = "Generation Plant")
central_transmission = pp.create_bus(net, vn_kv = 14.4, name = "Central Transmission Substation")
#wing buses
clusters = 6
houses_per_cluster = 30
distribution = [None]*clusters
transmission = [None]*clusters
house_bus = [None]*houses_per_cluster

pp.create_ext_grid(net, bus=central_transmission, vm_pu=1, name="Swing bus")
for cluster in range(clusters):
    transmission[cluster] = pp.create_bus(net, vn_kv = 14.4, name = "Transmission Substation - Cluster %s" \
                                           %cluster)
    distribution[cluster] = pp.create_bus(net, vn_kv = 0.415, name = "Distribution Substation - Cluster %s" \
                                           %cluster)
    if(cluster == 0):
            pp.create_line_from_parameters(net,from_bus= central_transmission, to_bus= transmission[cluster], \
                                       length_km=0.545, r_ohm_per_km = 0.916, x_ohm_per_km = 1.574,\
                                       c_nf_per_km = 1e-9, max_i_ka=0.242,\
                                       name = "Generation to cluster %s" %(cluster))
    else:
            pp.create_line_from_parameters(net,from_bus=transmission[cluster-1],\
                                      to_bus=transmission[cluster],length_km=0.015,\
                                      r_ohm_per_km = 0.916, x_ohm_per_km = 1.574,\
                                      c_nf_per_km = 1e-9, max_i_ka=0.242,\
                                      name = "cluster %s to cluster %s" %(cluster - 1, cluster) )
   
    
    pp.create_transformer_from_parameters(net, hv_bus =  transmission[cluster], lv_bus =  distribution[cluster], sn_mva=0.4,vn_hv_kv=14.4,vn_lv_kv=0.415,vkr_percent=1,vk_percent=1, pfe_kw=0.25, i0_percent=0.33, name="TD tx %s" %clusters)  
    

    for house in range(houses_per_cluster):
        house_bus[house] = pp.create_bus(net, vn_kv =0.415, name = "Cluster%s - House %s" \
                                           %(cluster,house))
        if(house == 0):
            pp.create_line_from_parameters(net,from_bus= distribution[cluster], to_bus= house_bus[house], \
                                       length_km=0.015, r_ohm_per_km = 0.979, x_ohm_per_km = 0.872,\
                                       c_nf_per_km = 1e-9, max_i_ka=0.184,\
                                       name = "District %s Substation to home %s" %(cluster, house))
        else:
            pp.create_line_from_parameters(net,from_bus=house_bus[house - 1],\
                                      to_bus=house_bus[house],length_km=0.015,\
                                      r_ohm_per_km = 0.979, x_ohm_per_km = 0.872,\
                                      c_nf_per_km = 1e-9, max_i_ka=0.184,\
                                      name = "Home %s to Home %s" %(house - 1, house) )
        pp.create_load(net, bus= house_bus[house], p_mw=p_demand, q_mvar=q_demand, \
                  name="Cluster %s House %s" %(cluster, house))
     


    
#school buses
Primary_school_transmission = pp.create_bus(net, vn_kv = 14.4, name = "Primary School transmission Substation")
Primary_school_distribution = pp.create_bus(net, vn_kv = .415, name = "Primary School Distribution Substation")
Primary_school_load = pp.create_bus(net, vn_kv = .415, name = "Primary School load")

#EVs station
EVs_station_transmission = pp.create_bus(net, vn_kv = 14.4, name = "EVs Transmission Substation")
EVs_station_distribution = pp.create_bus(net, vn_kv = .415, name = "EVS Distribution Substation")
EVs_station_load = pp.create_bus(net, vn_kv = .415, name = "EVs load")


#estate buses
estate_houses = 20
estate_transmission = pp.create_bus(net, vn_kv = 14.4, name = "Estate Transmission Substation")
estate_distribution = pp.create_bus(net, vn_kv = .415, name = "Estate Distribution Substation")
estate_house_bus = {}
for i in range (estate_houses):
    estate_house_bus[i] = pp.create_bus(net,vn_kv=.415,name="Estate_415_h{}".format(i))




net.bus

,name,vn_kv,type,zone,in_service
0,Generation Plant,69.000,b,None,True
1,Central Transmission Substation,14.400,b,None,True
2,Transmission Substation - Cluster 0,14.400,b,None,True
3,Distribution Substation - Cluster 0,0.415,b,None,True
4,Cluster0 - House 0,0.415,b,None,True
...,...,...,...,...,...
217,Estate_415_h15,0.415,b,None,True
218,Estate_415_h16,0.415,b,None,True
219,Estate_415_h17,0.415,b,None,True
220,Estate_415_h18,0.415,b,None,True


In [4]:
#Creating different loads
#creating the external Grid(swing bus)
pp.create_ext_grid(net, bus=central_transmission, vm_pu=1, name="Swing bus")

#Primary school load
pp.create_load(net, bus= Primary_school_distribution, p_mw=0.0125, q_mvar=0.0107, name="school Load_{}")
    
#EVs Charging station load
pp.create_load(net, bus= EVs_station_distribution, p_mw=0.0125, q_mvar=0.0107, name="EVs Charging Load_{}")
  
#Estate load
estate_house_load = {}
for i in range (estate_houses):
     estate_house_load[i]=pp.create_load(net, bus= estate_house_bus[i], p_mw=0.0108, q_mvar=0.00923, name="estate Load_{}".format(i))
        
    
net.load 

,name,bus,p_mw,q_mvar,const_z_percent,const_i_percent,sn_mva,scaling,in_service,type
0,Cluster 0 House 0,4,0.00108,0.000924,0.0,0.0,NaN,1.0,True,wye
1,Cluster 0 House 1,5,0.00108,0.000924,0.0,0.0,NaN,1.0,True,wye
2,Cluster 0 House 2,6,0.00108,0.000924,0.0,0.0,NaN,1.0,True,wye
3,Cluster 0 House 3,7,0.00108,0.000924,0.0,0.0,NaN,1.0,True,wye
4,Cluster 0 House 4,8,0.00108,0.000924,0.0,0.0,NaN,1.0,True,wye
...,...,...,...,...,...,...,...,...,...,...
197,estate Load_15,217,0.01080,0.009230,0.0,0.0,NaN,1.0,True,wye
198,estate Load_16,218,0.01080,0.009230,0.0,0.0,NaN,1.0,True,wye
199,estate Load_17,219,0.01080,0.009230,0.0,0.0,NaN,1.0,True,wye
200,estate Load_18,220,0.01080,0.009230,0.0,0.0,NaN,1.0,True,wye


In [5]:
#Creating Transformers
#Generation to transmission Transformer
pp.create_transformer_from_parameters(net, hv_bus=central_generation, lv_bus=central_transmission, sn_mva=0.4, vn_hv_kv=33, vn_lv_kv=14.4, vkr_percent=1, vk_percent=1, pfe_kw=1, i0_percent=1)

#school transformer
pp.create_transformer_from_parameters(net, hv_bus=Primary_school_transmission, lv_bus=Primary_school_distribution, sn_mva=0.4, vn_hv_kv=14.4, vn_lv_kv=0.415, vkr_percent=1, vk_percent=1, pfe_kw=1, i0_percent=1)
   

#EVs_station transformer
pp.create_transformer_from_parameters(net, hv_bus=EVs_station_transmission, lv_bus=EVs_station_distribution, sn_mva=0.4, vn_hv_kv=14.4, vn_lv_kv=0.415, vkr_percent=1, vk_percent=1, pfe_kw=1, i0_percent=1)
   

#Estate transformer
pp.create_transformer_from_parameters(net, hv_bus=estate_transmission, lv_bus=estate_distribution, sn_mva=0.4, vn_hv_kv=14.4, vn_lv_kv=0.415, vkr_percent=1, vk_percent=1, pfe_kw=1, i0_percent=1)
    
    
    
net.trafo

,name,std_type,hv_bus,lv_bus,sn_mva,vn_hv_kv,vn_lv_kv,vk_percent,vkr_percent,pfe_kw,...,tap_neutral,tap_min,tap_max,tap_step_percent,tap_step_degree,tap_pos,tap_phase_shifter,parallel,df,in_service
0,TD tx 6,None,2,3,0.4,14.4,0.415,1.0,1.0,0.25,...,NaN,NaN,NaN,NaN,NaN,NaN,False,1,1.0,True
1,TD tx 6,None,34,35,0.4,14.4,0.415,1.0,1.0,0.25,...,NaN,NaN,NaN,NaN,NaN,NaN,False,1,1.0,True
2,TD tx 6,None,66,67,0.4,14.4,0.415,1.0,1.0,0.25,...,NaN,NaN,NaN,NaN,NaN,NaN,False,1,1.0,True
3,TD tx 6,None,98,99,0.4,14.4,0.415,1.0,1.0,0.25,...,NaN,NaN,NaN,NaN,NaN,NaN,False,1,1.0,True
4,TD tx 6,None,130,131,0.4,14.4,0.415,1.0,1.0,0.25,...,NaN,NaN,NaN,NaN,NaN,NaN,False,1,1.0,True
5,TD tx 6,None,162,163,0.4,14.4,0.415,1.0,1.0,0.25,...,NaN,NaN,NaN,NaN,NaN,NaN,False,1,1.0,True
6,None,None,0,1,0.4,33.0,14.400,1.0,1.0,1.00,...,NaN,NaN,NaN,NaN,NaN,NaN,False,1,1.0,True
7,None,None,194,195,0.4,14.4,0.415,1.0,1.0,1.00,...,NaN,NaN,NaN,NaN,NaN,NaN,False,1,1.0,True
8,None,None,197,198,0.4,14.4,0.415,1.0,1.0,1.00,...,NaN,NaN,NaN,NaN,NaN,NaN,False,1,1.0,True
9,None,None,200,201,0.4,14.4,0.415,1.0,1.0,1.00,...,NaN,NaN,NaN,NaN,NaN,NaN,False,1,1.0,True


In [6]:
#creating the generators
wind_Generation = pp.create_sgen(net, bus = central_generation, p_mw = 3.75, q_mvar =  1.816, name = "wind farm Generator")


In [7]:
#Creating Lines
#central generation to central transmission
pp.create_line_from_parameters(net, from_bus =central_generation, to_bus =  central_transmission, length_km = 0.15, r_ohm_per_km = 0.0125 , x_ohm_per_km = 0.0813, c_nf_per_km = 289, max_i_ka = 0.2, name = "Generation to central transmission")


#central transmission to school transmission
pp.create_line_from_parameters(net, from_bus =central_transmission, to_bus =  Primary_school_transmission, length_km = 0.145, r_ohm_per_km = 0.0125, x_ohm_per_km = 0.0817, c_nf_per_km = 155, max_i_ka = 0.2, name = "central transmission to  school distribution" )
pp.create_line_from_parameters(net, from_bus =Primary_school_transmission, to_bus = Primary_school_distribution, length_km = 0.145, r_ohm_per_km = 0.0125, x_ohm_per_km = 0.0817, c_nf_per_km = 155, max_i_ka = 0.2, name = "central transmission to  school distribution" )
pp.create_line_from_parameters(net, from_bus =Primary_school_distribution, to_bus = Primary_school_load, length_km = 0.145, r_ohm_per_km = 0.0125, x_ohm_per_km = 0.0817, c_nf_per_km = 155, max_i_ka = 0.2, name = "central transmission to  school distribution" )
        
        

#central transmission to  EVs_station
pp.create_line_from_parameters(net, from_bus =central_transmission, to_bus =  EVs_station_transmission, length_km = 0.265, r_ohm_per_km = 0.0125, x_ohm_per_km = 0.0817, c_nf_per_km = 155, max_i_ka = 0.2, name = "central transmission to  EVs_station distribution" )
pp.create_line_from_parameters(net, from_bus =EVs_station_transmission, to_bus =  EVs_station_distribution, length_km = 0.265, r_ohm_per_km = 0.0125, x_ohm_per_km = 0.0817, c_nf_per_km = 155, max_i_ka = 0.2, name = "central transmission to  EVs_station distribution" )
pp.create_line_from_parameters(net, from_bus =EVs_station_distribution, to_bus =  EVs_station_load, length_km = 0.265, r_ohm_per_km = 0.0125, x_ohm_per_km = 0.0817, c_nf_per_km = 155, max_i_ka = 0.2, name = "central transmission to  EVs_station distribution" )



   
#central transmission to Estate
pp.create_line_from_parameters(net, from_bus =central_transmission, to_bus =  estate_transmission, length_km = 0.045, r_ohm_per_km =0.0125, x_ohm_per_km = 0.0817, c_nf_per_km = 155, max_i_ka = 0.2, name = "central transmission to  estate distribution" )
pp.create_line_from_parameters(net, from_bus =estate_transmission, to_bus =  estate_distribution, length_km = 0.045, r_ohm_per_km =0.0125, x_ohm_per_km = 0.0817, c_nf_per_km = 155, max_i_ka = 0.2, name = "central transmission to  estate distribution" )



#estate houses
for i in range (estate_houses):
    pp.create_line_from_parameters(net, from_bus =estate_distribution, to_bus = estate_house_bus[i]  , length_km = 0.080, r_ohm_per_km =0.0125, x_ohm_per_km = 0.0955, c_nf_per_km = 1e-9, max_i_ka = 0.2, name = "Estate distribution to  estate houses %s" %(estate_houses) )

net.line

,name,std_type,from_bus,to_bus,length_km,r_ohm_per_km,x_ohm_per_km,c_nf_per_km,g_us_per_km,max_i_ka,df,parallel,type,in_service
0,Generation to cluster 0,None,1,2,0.545,0.9160,1.5740,1.000000e-09,0.0,0.242,1.0,1,None,True
1,District 0 Substation to home 0,None,3,4,0.015,0.9790,0.8720,1.000000e-09,0.0,0.184,1.0,1,None,True
2,Home 0 to Home 1,None,4,5,0.015,0.9790,0.8720,1.000000e-09,0.0,0.184,1.0,1,None,True
3,Home 1 to Home 2,None,5,6,0.015,0.9790,0.8720,1.000000e-09,0.0,0.184,1.0,1,None,True
4,Home 2 to Home 3,None,6,7,0.015,0.9790,0.8720,1.000000e-09,0.0,0.184,1.0,1,None,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
210,Estate distribution to estate houses 20,None,201,217,0.080,0.0125,0.0955,1.000000e-09,0.0,0.200,1.0,1,None,True
211,Estate distribution to estate houses 20,None,201,218,0.080,0.0125,0.0955,1.000000e-09,0.0,0.200,1.0,1,None,True
212,Estate distribution to estate houses 20,None,201,219,0.080,0.0125,0.0955,1.000000e-09,0.0,0.200,1.0,1,None,True
213,Estate distribution to estate houses 20,None,201,220,0.080,0.0125,0.0955,1.000000e-09,0.0,0.200,1.0,1,None,True


In [8]:
pp.runpp(net, algorithm='nr')

In [9]:
net.res_line

,p_from_mw,q_from_mvar,p_to_mw,q_to_mvar,pl_mw,ql_mvar,i_from_ka,i_to_ka,i_ka,vm_from_pu,va_from_degree,vm_to_pu,va_to_degree,loading_percent
0,0.207631,0.184332,-0.207446,-0.184013,0.000186,0.000319,0.011132,0.011132,0.011132,1.000000,0.000000,0.998738,-0.023818,4.599999
1,0.034271,0.029375,-0.034097,-0.029219,0.000174,0.000155,0.062929,0.062929,0.062929,0.997876,0.019329,0.992712,0.013654,34.200714
2,0.033017,0.028296,-0.032853,-0.028150,0.000164,0.000146,0.060938,0.060938,0.060938,0.992712,0.013654,0.987711,0.008111,33.118370
3,0.031773,0.027226,-0.031620,-0.027090,0.000153,0.000136,0.058936,0.058936,0.058936,0.987711,0.008111,0.982875,0.002705,32.030547
4,0.030540,0.026167,-0.030397,-0.026039,0.000143,0.000127,0.056925,0.056925,0.056925,0.982875,0.002705,0.978203,-0.002557,30.937370
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
210,0.010801,0.009239,-0.010800,-0.009230,0.000001,0.000009,0.019774,0.019774,0.019774,0.999992,-0.000382,0.999520,-0.024773,9.887031
211,0.010801,0.009239,-0.010800,-0.009230,0.000001,0.000009,0.019774,0.019774,0.019774,0.999992,-0.000382,0.999520,-0.024773,9.887031
212,0.010801,0.009239,-0.010800,-0.009230,0.000001,0.000009,0.019774,0.019774,0.019774,0.999992,-0.000382,0.999520,-0.024773,9.887031
213,0.010801,0.009239,-0.010800,-0.009230,0.000001,0.000009,0.019774,0.019774,0.019774,0.999992,-0.000382,0.999520,-0.024773,9.887031


In [10]:
net.res_load

,p_mw,q_mvar
0,0.00108,0.000924
1,0.00108,0.000924
2,0.00108,0.000924
3,0.00108,0.000924
4,0.00108,0.000924
...,...,...
197,0.01080,0.009230
198,0.01080,0.009230
199,0.01080,0.009230
200,0.01080,0.009230


In [11]:
net.res_trafo

,p_hv_mw,q_hv_mvar,p_lv_mw,q_lv_mvar,pl_mw,ql_mvar,i_hv_ka,i_lv_ka,vm_hv_pu,va_hv_degree,vm_lv_pu,va_lv_degree,loading_percent
0,0.034573,0.030666,-0.034271,-0.029375,0.000301,0.001292,0.001855,0.062929,0.998738,-0.023818,0.997876,0.019329,11.568045
1,0.034573,0.030667,-0.034271,-0.029375,0.000301,0.001292,0.001855,0.062931,0.998709,-0.024365,0.997847,0.018785,11.568408
2,0.034573,0.030667,-0.034272,-0.029375,0.000301,0.001292,0.001855,0.062933,0.998686,-0.024802,0.997823,0.018349,11.568698
3,0.034573,0.030667,-0.034272,-0.029375,0.000301,0.001292,0.001855,0.062934,0.998668,-0.025131,0.997806,0.018023,11.568916
4,0.034573,0.030667,-0.034272,-0.029375,0.000301,0.001292,0.001855,0.062935,0.998657,-0.025349,0.997795,0.017805,11.569061
5,0.034573,0.030667,-0.034272,-0.029375,0.000301,0.001291,0.001855,0.062935,0.998651,-0.025459,0.997789,0.017696,11.569134
6,91.233360,0.025059,-43.633097,-0.015809,47.600262,0.009250,0.763408,1.749415,0.999970,-0.012881,1.000000,0.000000,10908.661894
7,0.000532,0.001911,0.000469,0.001962,0.001000,0.003873,0.000080,0.002806,0.999999,-0.000038,0.999998,-0.000075,0.504291
8,0.000555,0.001889,0.000446,0.001983,0.001000,0.003873,0.000079,0.002828,0.999999,-0.000072,0.999997,-0.000139,0.508218
9,0.000656,0.001803,0.000344,0.002070,0.001000,0.003873,0.000077,0.002919,0.999996,-0.000191,0.999992,-0.000382,0.524523


In [12]:
net.res_bus

,vm_pu,va_degree,p_mw,q_mvar
0,0.999970,-0.012881,-3.750000,-1.816000
1,1.000000,0.000000,-44.304933,1.473839
2,0.998738,-0.023818,0.000000,0.000000
3,0.997876,0.019329,0.000000,0.000000
4,0.992712,0.013654,0.001080,0.000924
...,...,...,...,...
217,0.999520,-0.024773,0.010800,0.009230
218,0.999520,-0.024773,0.010800,0.009230
219,0.999520,-0.024773,0.010800,0.009230
220,0.999520,-0.024773,0.010800,0.009230
